# Spark DataFrame Basics

In [2]:
######### Basic Imports ##################
from pyspark.sql import SparkSession

In [3]:
# Start the Spark Session
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [3]:
# Read a dataset
df = spark.read.json('people.json')

In [4]:
# Show the data
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [5]:
# Look at the schema
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [6]:
# Just want the column names
df.columns

['age', 'name']

In [9]:
# To get statistical summary of dataframe
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



#### Create Custom Schema

In [13]:
# Sometimes have to correct the schema

# import type tools to deal with schema changes
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [14]:
# Create list of structure field
# Name of the field
# the type of data it will carry
# Can it be null
data_schema = [StructField('age',IntegerType(), True),
              StructField('name', StringType(), True)]

In [15]:
# Create Schema
final_struct = StructType(fields=data_schema)

In [17]:
# Now import the data with the schema
df = spark.read.json('people.json', schema=final_struct)

In [19]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



#### Grabbing Data

In [21]:
# Need to use the select method to get data
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [22]:
# Getting head and can index what is returned
df.head(2)[0]

Row(age=None, name='Michael')

In [23]:
# Select multiple columns
df.select(['age', 'name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [25]:
# Creating new columns (have to give it a column)
df.withColumn('doubleage', df['age']*2).show()

+----+-------+---------+
| age|   name|doubleage|
+----+-------+---------+
|null|Michael|     null|
|  30|   Andy|       60|
|  19| Justin|       38|
+----+-------+---------+



In [26]:
# Can use pure sql to directly act with the dataframe

# Have to register dataframe as a sql temporary view
df.createOrReplaceTempView('people')

In [30]:
new_results = spark.sql("SELECT * FROM people WHERE age=30")

In [31]:
new_results.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



# Basic Operations

In [4]:
df = spark.read.csv('appl_stock.csv', inferSchema=True, header=True)

In [5]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [6]:
df.show()

+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|               Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:00|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:00|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:00|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:00|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|    

In [11]:
# Can filter and can do select to select specific columns This is SQL like.
df.filter("Close < 200").select(['Open','Close']).show()

+------------------+------------------+
|              Open|             Close|
+------------------+------------------+
|206.78000600000001|            197.75|
|        204.930004|        199.289995|
|        201.079996|        192.060003|
|192.36999699999998|        194.729998|
|        195.909998|        195.859997|
|        195.169994|        199.229994|
|        196.730003|        192.050003|
|192.63000300000002|        195.460001|
|        195.690006|194.11999699999998|
|        196.419996|196.19000400000002|
|        195.889997|195.12000700000002|
|        194.880001|        198.669994|
|        199.999998|        197.059998|
|         92.699997|         93.699997|
|         94.730003|             94.25|
|         94.129997|         93.860001|
|         94.040001|         92.290001|
|         92.199997|         91.279999|
|         91.510002|         92.199997|
|         92.309998| 92.08000200000001|
+------------------+------------------+
only showing top 20 rows



In [12]:
# Will use Python comparison instead
df.filter(df['Close'] < 500).show()

+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|               Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:00|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:00|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:00|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:00|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|    

In [14]:
# Lets filter off of multiple conditions. Need to use & for and and | for or
df.filter( (df['Close'] < 200) & (df['Open']> 200)).show()

+-------------------+------------------+----------+----------+----------+---------+------------------+
|               Date|              Open|      High|       Low|     Close|   Volume|         Adj Close|
+-------------------+------------------+----------+----------+----------+---------+------------------+
|2010-01-22 00:00:00|206.78000600000001|207.499996|    197.16|    197.75|220441900|         25.620401|
|2010-01-28 00:00:00|        204.930004|205.500004|198.699995|199.289995|293375600|25.819922000000002|
|2010-01-29 00:00:00|        201.079996|202.199995|190.250002|192.060003|311488100|         24.883208|
+-------------------+------------------+----------+----------+----------+---------+------------------+



In [17]:
# Can get specific row instead of show, do collect. Will get the real data so can manipualte it. 
result = df.filter(df['Low']==197.16).collect()

In [19]:
row = result[0]
row.asDict()

{'Date': datetime.datetime(2010, 1, 22, 0, 0),
 'Open': 206.78000600000001,
 'High': 207.499996,
 'Low': 197.16,
 'Close': 197.75,
 'Volume': 220441900,
 'Adj Close': 25.620401}

# Group By and Aggregate Functions

In [20]:
df = spark.read.csv('sales_info.csv', inferSchema=True, header=True)
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [21]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [22]:
# Lets group by company
df.groupby("Company").mean().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [23]:
# Can do aggregations without group by. If just want to get total across all rows. 
df.agg({'Sales':'sum'}).show()

+----------+
|sum(Sales)|
+----------+
|    4327.0|
+----------+



In [26]:
# Import functions from spark
from pyspark.sql.functions import countDistinct, avg, stddev

In [29]:
# Can use the functions and add an alisas to title the column
df.select(countDistinct('Sales').alias('Count')).show()

+-----+
|Count|
+-----+
|   11|
+-----+



In [38]:
df.select(stddev("Sales")).show()

+------------------+
|stddev_samp(Sales)|
+------------------+
|250.08742410799007|
+------------------+



In [41]:
# Can also format the number
from pyspark.sql.functions import format_number

sales_std = df.select(stddev("Sales").alias('std'))
# Call the format number
sales_std.select(format_number('std',2)).show()

+---------------------+
|format_number(std, 2)|
+---------------------+
|               250.09|
+---------------------+



In [37]:
# OrderBy
# Ascending
df.orderBy("Sales").show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [42]:
# Descending call off the column itself.
df.orderBy(df["Sales"].desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+



# Missing Data